# Lab 2: Debug a Broken Vanilla GAN (find 12+ issues)

In [1]:
import torch, torchvision, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

In [3]:


device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(0.1307,0.3081)])  # normalizes images to improve model stability. For MNIST, mean = 0.1307 and std = 0.3081.z
loader=DataLoader(torchvision.datasets.MNIST('./data',True,download=True,transform=transform),batch_size=256,shuffle=True,num_workers=2,pin_memory=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 501kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.62MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.2MB/s]


In [12]:
z_dim=100; g_lr=2e-2; d_lr=2e-5
class D(nn.Module):
    def __init__(self):
        super().__init__(); self.net=nn.Sequential(nn.Conv2d(1,32,4,2,1),
                                                   nn.BatchNorm2d(32),
                                                   nn.LeakyReLU(0.2,True),
                                                   nn.Conv2d(32,64,4,2,1),
                                                   nn.BatchNorm2d(64),
                                                   nn.LeakyReLU(0.2,True),
                                                   nn.Conv2d(64,1,7,1,0))
    def forward(self,x):return self.net(x).view(x.size(0),1)

class G(nn.Module):
    def __init__(self):
        super().__init__(); self.net=nn.Sequential(nn.ConvTranspose2d(100,128,4,1,0),
                                                   nn.BatchNorm2d(128),
                                                   nn.ReLU(True),
                                                   nn.ConvTranspose2d(128,64,4,2,1),
                                                   nn.BatchNorm2d(64),
                                                   nn.ReLU(True),
                                                   nn.ConvTranspose2d(64,32,4,2,1),
                                                   nn.BatchNorm2d(32),
                                                   nn.ReLU(True),
                                                   nn.ConvTranspose2d(32,1,3,2,2),
                                                   nn.Tanh())      # Tanh was added at the end to normalize the output to the range of [-1, 1].
    def forward(self,z):  return self.net(z)       # Reshape Issue in Generator
Dnet=D().to(device); Gnet=G().to(device)
crit=nn.BCEWithLogitsLoss(); opt_d=torch.optim.Adam(Dnet.parameters(),lr=d_lr,betas=(0.9,0.999)); opt_g=torch.optim.Adam(Gnet.parameters(),lr=g_lr,betas=(0.9,0.999))

In [ ]:
from tqdm import tqdm
for epoch in range(10):
 for real,_ in tqdm(loader):
    real=real.to(device)
    b=real.size(0)
    z=torch.randn(b,z_dim,device=device)
    fake=Gnet(z.view(b,z_dim,1,1))
    loss_d=crit(Dnet(real),torch.zeros(b,1,device=device))+crit(Dnet(fake),torch.ones(b,1,device=device))  # BUG labels #
    loss_d.backward(); opt_d.step()  # BUG
    z=torch.randn(b,z_dim,device=device); fake=Gnet(z.view(b,z_dim,1,1))
    loss_g=crit(Dnet(fake),torch.zeros(b,1,device=device))  # BUG non-sat
    loss_g.backward()  # BUG missing step
print('Now fix all the issues.')

  0%|          | 0/235 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
 16%|█▌        | 38/235 [00:32<02:40,  1.23it/s]